In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# [1] 파일 경로
PARQUET_PATH = "/content/drive/MyDrive/LSTMAE_PROJECT/all_kospi_data.parquet"
EVENT_CSV_PATH = "/content/drive/MyDrive/LSTMAE_PROJECT/event_detected.csv"
SELECTED_CSV_PATH = "/content/drive/MyDrive/LSTMAE_PROJECT/random_selected_stocks_sets_nodup.csv"
OUTPUT_CSV_PATH = "/content/drive/MyDrive/LSTMAE_PROJECT/filtered_kospi_data_300.csv"

# [2] 파일 확인 및 로딩
if not os.path.exists(PARQUET_PATH) or not os.path.exists(EVENT_CSV_PATH) or not os.path.exists(SELECTED_CSV_PATH):
    print("파일이 존재하지 않습니다.")
    exit()

df = pd.read_parquet(PARQUET_PATH)
event_df = pd.read_csv(EVENT_CSV_PATH)
selected_stocks_df = pd.read_csv(SELECTED_CSV_PATH)

df["날짜"] = pd.to_datetime(df["날짜"])
event_df["날짜"] = pd.to_datetime(event_df["날짜"])

# [3] 종목명 정규화 함수
def clean_name(name):
    return str(name).strip().replace("\u3000", "").replace("\xa0", "")

df["종목명"] = df["종목명"].apply(clean_name)
event_df["종목명"] = event_df["종목명"].apply(clean_name)

# [4] 300개 종목명(중복 제거) 리스트 추출
selected_stocks_flat = []
for col in selected_stocks_df.columns:
    selected_stocks_flat.extend(selected_stocks_df[col].dropna().tolist())
selected_stocks_flat = [clean_name(s) for s in selected_stocks_flat]
selected_stocks = list(dict.fromkeys(selected_stocks_flat))  # 순서 유지하며 중복 제거

print(f"총 {len(selected_stocks)}개 종목 처리")

# [5] 이벤트 맵 생성
event_map = event_df.groupby("종목명")["날짜"].apply(list).to_dict()

# [6] 처리
result_list = []

print("\n[📊 종목별 처리 결과]")
print("-" * 60)

for stock in selected_stocks:
    stock_df = df[df["종목명"] == stock].copy()
    stock_df.sort_values("날짜", inplace=True)

    if stock not in event_map:
        print(
            f"[{stock}] 이벤트 없음 → 전체 유지 ({stock_df['날짜'].min().date()} ~ {stock_df['날짜'].max().date()}) / {len(stock_df)}일"
        )
        result_list.append(stock_df)
        continue

    # 이벤트가 있는 종목 → 여러 이벤트 중 손실 가장 적은 쪽 선택
    best_df = None
    best_len = -1
    best_event_date = None

    for event_date in event_map[stock]:
        window_start = event_date - pd.Timedelta(days=5)
        window_end = event_date + pd.Timedelta(days=5)

        before_df = stock_df[stock_df["날짜"] < window_start]
        after_df = stock_df[stock_df["날짜"] > window_end]

        print(f"  🔍 [{stock}] 이벤트 날짜: {event_date.date()}")
        print(f"     ➤ 제거 구간: {window_start.date()} ~ {window_end.date()}")
        print(f"     ➤ before: {len(before_df)}일")
        print(f"     ➤ after : {len(after_df)}일")

        if len(before_df) >= len(after_df):
            candidate_df = before_df
            chosen = "before"
        else:
            candidate_df = after_df
            chosen = "after"

        print(f"     ➤ 선택된 구간: {chosen} ({len(candidate_df)}일)")

        if len(candidate_df) > best_len:
            best_df = candidate_df
            best_len = len(candidate_df)
            best_event_date = event_date

    result_list.append(best_df)
    print(
        f"✅ [{stock}] 최종 선택 날짜: {best_event_date.date()} → {best_df['날짜'].min().date()} ~ {best_df['날짜'].max().date()} / {len(best_df)}일"
    )

# [7] 저장
final_df = (
    pd.concat(result_list).sort_values(by=["종목코드", "날짜"]).reset_index(drop=True)
)
final_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")

print("\n[✅] 최종 결과 저장 완료:", OUTPUT_CSV_PATH)


총 300개 종목 처리

[📊 종목별 처리 결과]
------------------------------------------------------------
[쌍용양회(3우B)] 이벤트 없음 → 전체 유지 (2005-01-03 ~ 2008-08-01) / 927일
[우성넥스티어] 이벤트 없음 → 전체 유지 (2005-01-03 ~ 2007-07-23) / 663일
[지엔비씨더스] 이벤트 없음 → 전체 유지 (2008-04-25 ~ 2008-12-18) / 165일
[코오롱(1우)] 이벤트 없음 → 전체 유지 (2005-01-03 ~ 2006-12-19) / 511일
  🔍 [코오롱글로벌우] 이벤트 날짜: 2022-12-22
     ➤ 제거 구간: 2022-12-17 ~ 2022-12-27
     ➤ before: 2693일
     ➤ after : 621일
     ➤ 선택된 구간: before (2693일)
  🔍 [코오롱글로벌우] 이벤트 날짜: 2023-01-30
     ➤ 제거 구간: 2023-01-25 ~ 2023-02-04
     ➤ before: 2717일
     ➤ after : 596일
     ➤ 선택된 구간: before (2717일)
✅ [코오롱글로벌우] 최종 선택 날짜: 2023-01-30 → 2012-01-11 ~ 2023-01-20 / 2717일
[코오롱유화] 이벤트 없음 → 전체 유지 (2005-01-03 ~ 2007-06-14) / 636일
[하나니켈2호] 이벤트 없음 → 전체 유지 (2008-02-01 ~ 2020-06-29) / 3072일
[하나증권(1우)] 이벤트 없음 → 전체 유지 (2005-01-03 ~ 2006-10-25) / 472일
  🔍 [한국수출포장] 이벤트 날짜: 2023-04-14
     ➤ 제거 구간: 2023-04-09 ~ 2023-04-19
     ➤ before: 709일
     ➤ after : 544일
     ➤ 선택된 구간: before (709일)
✅ [한국수출포장] 최종 선택